In [6]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import csv
page = 1
stop = False
cutoff_date = datetime.strptime("01.01.2020", "%d.%m.%Y").date()
    
with open("kztgold.csv", mode="w", newline="",encoding="utf8") as file:
    writer = csv.writer(file)
    writer.writerow(["date", "gold_kzt"])

    while not stop:
        print(f"Обработка страницы {page}...", end="\r")
            
        response = requests.get(f"https://www.nationalbank.kz/ru/gold/zoloto?from=01.01.2020&to=10.10.2025&page={page}")
        if response.status_code != 200:
            print(f"Ошибка {response.status_code}")

        html_content = response.text
        soup = BeautifulSoup(html_content, "html.parser")

        table_div = soup.find("div", class_="table-responsive")
        if not table_div:
            stop = True
            break

        table = table_div.find("table")
        rows = table.find_all("tr")

        for row in rows[1:]:
            cols = row.find_all("td")
            if len(cols) < 4:
                continue

            date_str = cols[2].text.strip()
            gold_kzt = cols[3].text.strip()

            date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()
            if date_obj < cutoff_date:
                    print("Все данные получены!")   
                    stop = True
                    break
            writer.writerow([date_obj.strftime("%d.%m.%Y"), gold_kzt])
        page += 1

In [ ]:
import pandas as pd
df = pd.read_csv("kztgold.csv")
df

,date,gold_kzt
0,10-10-2025,70 350.54
1,09-10-2025,70 056.70
2,08-10-2025,68 998.73
3,07-10-2025,69 156.38
4,06-10-2025,67 966.79
...,...,...
1415,10-01-2020,18 736.35
1416,09-01-2020,19 247.03
1417,08-01-2020,19 198.81
1418,06-01-2020,19 011.49


In [25]:
import pandas as pd
df = pd.read_csv("kztgold.csv")
# df
# df['gold_kzt'] = df['gold_kzt'].astype(str).str.replace(" ", "").astype(float)
# df['gold_kzt_variability'] = df['gold_kzt'].pct_change() * 100
df['gold_kzt_variability'] = df['gold_kzt_variability'].round(2)
df.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0'], axis=1,inplace=True)
df.to_csv("kztgold.csv",index=False)
df

,date,gold_kzt,gold_kzt_variability
0,10-10-2025,70350.54,NaN
1,09-10-2025,70056.70,-0.42
2,08-10-2025,68998.73,-1.51
3,07-10-2025,69156.38,0.23
4,06-10-2025,67966.79,-1.72
...,...,...,...
1415,10-01-2020,18736.35,-0.32
1416,09-01-2020,19247.03,2.73
1417,08-01-2020,19198.81,-0.25
1418,06-01-2020,19011.49,-0.98
